# STEP 1: Setup

In [1]:
import pandas as pd
import sqlite3

# STEP 2: Load CSV

In [2]:
orders = pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


# STEP 3: Load JSON

In [4]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
users.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

# STEP 4: Load SQL

## 4.1 Create SQLite connection

In [6]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

## 4.2 Run the SQL file & load restaurant table into pandas

In [7]:
cursor.execute("DROP TABLE IF EXISTS restaurants;")
conn.commit()

In [8]:
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

In [9]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [10]:
restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

# STEP 5: Merge the Data

## 4.1 Merge orders + users (LEFT JOIN)

In [11]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

## 4.2 Merge with restaurants (LEFT JOIN)

In [12]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)

In [13]:
final_df = final_df.drop(columns=['restaurant_name_x'])
final_df = final_df.rename(columns={'restaurant_name_y': 'restaurant_name'})

In [14]:
final_df.shape
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
0,1,2508,450,18-02-2023,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


# STEP 5: Save Final Dataset

In [15]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [16]:
final_df.isnull().sum()
final_df.describe()
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   name             10000 non-null  object 
 6   city             10000 non-null  object 
 7   membership       10000 non-null  object 
 8   restaurant_name  10000 non-null  object 
 9   cuisine          10000 non-null  object 
 10  rating           10000 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 859.5+ KB


## 1).City with highest total revenue from Gold members

In [17]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

## 2)Cuisine with highest average order value

In [18]:
final_df.groupby('cuisine')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

## 3).Distinct users with total spend > ₹1000

In [19]:
user_spend = final_df.groupby('user_id')['total_amount'].sum()
user_spend[user_spend > 1000].count()

np.int64(2544)

## 4)Restaurant rating range with highest revenue

In [20]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

In [21]:
final_df.groupby('rating_range')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

C:\Users\komal\AppData\Local\Temp\ipykernel_10936\156399170.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby('rating_range')['total_amount'] \


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

## 5)Among Gold members, city with highest average order value

In [22]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

## 6)Cuisine with lowest distinct restaurants but high revenue

In [23]:
restaurant_count = final_df.groupby('cuisine')['restaurant_id'].nunique()
revenue = final_df.groupby('cuisine')['total_amount'].sum()

summary = pd.DataFrame({
    'restaurant_count': restaurant_count,
    'revenue': revenue
}).sort_values(['restaurant_count', 'revenue'], ascending=[True, False])

summary

,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


## 7)% of orders placed by Gold members

In [24]:
gold_orders = final_df[final_df['membership'] == 'Gold'].shape[0]
total_orders = final_df.shape[0]

round((gold_orders / total_orders) * 100)

50

## 8)Restaurant with highest avg order value but < 20 orders

In [25]:
restaurant_stats = final_df.groupby('restaurant_name').agg(
    avg_order_value=('total_amount', 'mean'),
    order_count=('order_id', 'count')
)

restaurant_stats[restaurant_stats['order_count'] < 20] \
    .sort_values('avg_order_value', ascending=False)

,avg_order_value,order_count
restaurant_name,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
...,...,...
Restaurant_184,621.828947,19
Restaurant_498,596.815556,18
Restaurant_192,589.972857,14


## 9)Restaurant has the highest average order value but less than 20 total orders

In [26]:
final_df.groupby(['membership', 'cuisine'])['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

## 10)Quarter with highest total revenue

In [27]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')

final_df.groupby('quarter')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

C:\Users\komal\AppData\Local\Temp\ipykernel_10936\412655225.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

## 11)Total orders were placed by Gold members

In [28]:
final_df[final_df['membership'] == 'Gold'].shape[0]

4987

## 12)Total revenue from Hyderabad

In [29]:
round(
    final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
)

1889367

## 13)distinct users placed at least one order

In [30]:
final_df['user_id'].nunique()

2883

## 14)Average order value for Gold members 

In [31]:
round(
    final_df[final_df['membership'] == 'Gold']['total_amount'].mean(),
    2
)

np.float64(797.15)

## 15)orders were placed for restaurants with rating ≥ 4.5

In [32]:
final_df[final_df['rating'] >= 4.5].shape[0]

3374

## 16)Orders placed in the top revenue city among Gold members only

In [33]:
top_city = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

top_city

'Chennai'

In [34]:
final_df[
    (final_df['membership'] == 'Gold') &
    (final_df['city'] == top_city)
].shape[0]

1337

## 17)Column used to join orders.csv and users.json

In [35]:
set(orders.columns).intersection(set(users.columns))

{'user_id'}

## 18)Dataset containing cuisine & rating is stored in which format

In [36]:
type("restaurants.sql")

str

## 19)Total number of rows in final merged dataset

In [37]:
final_df.shape[0]

10000

## 20)If a user has no match in users.json, merged values will be?

In [38]:
final_df.isnull().sum()

order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
name                 0
city                 0
membership           0
restaurant_name      0
cuisine              0
rating               0
rating_range       331
quarter              0
dtype: int64

## 21)Pandas function used to combine datasets based on a key

In [39]:
pd.merge(orders, users, on='user_id', how='left')

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


## 22)membership column comes from which file

In [40]:
users.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

## 23)Join key for orders and restaurant details

In [41]:
set(orders.columns).intersection(set(restaurants.columns))

{'restaurant_id', 'restaurant_name'}

## 24)Column that identifies food type of restaurant

In [42]:
restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

## 25)If a user places multiple orders, how many times do details appear?

In [43]:
final_df['user_id'].value_counts().head()

user_id
2973    13
1515    12
1337    11
496     11
874     11
Name: count, dtype: int64